In [20]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:////tmp/mlflow.db"

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [21]:
client.list_experiments()

[<Experiment: artifact_location='./mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='homework', tags={}>,
 <Experiment: artifact_location='./mlruns/2', experiment_id='2', lifecycle_stage='active', name='random-forest-hyperopt', tags={}>,
 <Experiment: artifact_location='./mlruns/3', experiment_id='3', lifecycle_stage='active', name='my-cool-experiment', tags={}>,
 <Experiment: artifact_location='./mlruns/4', experiment_id='4', lifecycle_stage='active', name='random-forest-best-models', tags={}>]

In [22]:
client.create_experiment(name="my-cool-experiment")

MlflowException: Experiment(name=my-cool-experiment) already exists. Error: (raised as a result of Query-invoked autoflush; consider using a session.no_autoflush block if this flush is occurring prematurely)
(sqlite3.IntegrityError) UNIQUE constraint failed: experiments.name
[SQL: INSERT INTO experiments (name, artifact_location, lifecycle_stage) VALUES (?, ?, ?)]
[parameters: ('my-cool-experiment', None, 'active')]
(Background on this error at: https://sqlalche.me/e/14/gkpj)

In [ ]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='2',
    filter_string="metrics.rmse < 6.8",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [ ]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 013ed19c4b7449e492fbdd2f8492137d, rmse: 6.6284
run id: ff5568e6318749779db178f43b3152a0, rmse: 6.6297
run id: 2f93892ad2c844e9af449c608d205ec1, rmse: 6.6299
run id: 6adcf110dc264106a273c4bce84d1fa2, rmse: 6.6299
run id: 2fe0f89107564e2199b1e821bd46d2c0, rmse: 6.6299


In [ ]:
import mlflow
mlflow.set_tracking_uri("http://127.0.0.1:5001")

In [ ]:
run_id="013ed19c4b7449e492fbdd2f8492137d"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(
    model_uri=model_uri,
    name="just_test"
)

Registered model 'just_test' already exists. Creating a new version of this model...
2022/05/30 11:15:16 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: just_test, version 2
Created version '2' of model 'just_test'.


<ModelVersion: creation_timestamp=1653930916951, current_stage='None', description='', last_updated_timestamp=1653930916951, name='just_test', run_id='013ed19c4b7449e492fbdd2f8492137d', run_link='', source='./mlruns/2/013ed19c4b7449e492fbdd2f8492137d/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='2'>

In [ ]:
client.list_registered_models()

[<RegisteredModel: creation_timestamp=1653930991552, description='', last_updated_timestamp=1653930991552, latest_versions=[], name='hw', tags={}>,
 <RegisteredModel: creation_timestamp=1653927408668, description='', last_updated_timestamp=1653930916951, latest_versions=[<ModelVersion: creation_timestamp=1653927408708, current_stage='Archived', description='', last_updated_timestamp=1653929758711, name='just_test', run_id='aba9236df3754605bbf53c41afd08848', run_link='', source='./mlruns/1/aba9236df3754605bbf53c41afd08848/artifacts/model', status='READY', status_message=None, tags={'model': 'sklearn'}, user_id=None, version=1>,
  <ModelVersion: creation_timestamp=1653930916951, current_stage='None', description='', last_updated_timestamp=1653930916951, name='just_test', run_id='013ed19c4b7449e492fbdd2f8492137d', run_link='', source='./mlruns/2/013ed19c4b7449e492fbdd2f8492137d/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>], name='just_test', tag

In [ ]:
model_name="just_test"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: Archived
version: 2, stage: Staging


In [ ]:
client.transition_model_version_stage(
    name = model_name,
    version=2,
    stage="Staging",
    archive_existing_versions=False
)

<ModelVersion: creation_timestamp=1653930916951, current_stage='Staging', description='', last_updated_timestamp=1653931238030, name='just_test', run_id='013ed19c4b7449e492fbdd2f8492137d', run_link='', source='./mlruns/2/013ed19c4b7449e492fbdd2f8492137d/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [29]:
best_run = client.search_runs(
    experiment_ids="4",
    filter_string=""
)

In [30]:
best_run

[<Run: data=<RunData: metrics={'test_rmse': 6.548852722401935,
  'training_mae': 3.7776220690634528,
  'training_mse': 30.58526647604336,
  'training_r2_score': 0.771248028190807,
  'training_rmse': 5.530394784827152,
  'training_score': 0.771248028190807,
  'valid_rmse': 6.629893219637507}, params={'bootstrap': 'True',
  'ccp_alpha': '0.0',
  'criterion': 'squared_error',
  'max_depth': '20',
  'max_features': '1.0',
  'max_leaf_nodes': 'None',
  'max_samples': 'None',
  'min_impurity_decrease': '0.0',
  'min_samples_leaf': '2',
  'min_samples_split': '2',
  'min_weight_fraction_leaf': '0.0',
  'n_estimators': '25',
  'n_jobs': 'None',
  'oob_score': 'False',
  'random_state': '42',
  'verbose': '0',
  'warm_start': 'False'}, tags={'estimator_class': 'sklearn.ensemble._forest.RandomForestRegressor',
  'estimator_name': 'RandomForestRegressor',
  'mlflow.log-model.history': '[{"run_id": "e3db6ad9799644a8b6124e10a5bd3f99", '
                              '"artifact_path": "model", "utc_

In [ ]:
from datetime import datetime
date = 
model_version=2
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f""
)